In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlretrieve
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoAlertPresentException, TimeoutException, UnexpectedAlertPresentException, ElementClickInterceptedException
import time
import re
import base64

In [ ]:
options = ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
options.add_argument("--incognito")
driver = Chrome(options=options)
url = 'https://kupis.konkuk.ac.kr/sugang/login/loginTop.jsp'
driver.get(url)

time.sleep(0.5)

driver.find_element_by_name('stdNo').clear()
driver.find_element_by_name('stdNo').send_keys('id')
driver.find_element_by_name('pwd').clear()
driver.find_element_by_name('pwd').send_keys('pw')
driver.find_element_by_css_selector('form input[type=button]').click()

time.sleep(1)

driver.close()
driver.switch_to.window(driver.window_handles[0])
driver.get('https://kupis.konkuk.ac.kr/sugang/acd/cour/aply/courLessinApplyReg.jsp')

In [ ]:
# subj = [] 수강신청 할 과목 리스트
while True:
    for _ in subj:
        try:
            driver.find_element_by_name('strSbjtId').clear()
            driver.find_element_by_name('strSbjtId').send_keys(_)
            driver.find_element_by_name('new').click()
            try:
                WebDriverWait(driver, 3).until(EC.alert_is_present(),
                                                       'Timed out waiting for PA creation ' +
                                                       'confirmation popup to appear.')

                alert = Alert(driver)

                if alert.text == '입력하신 과목번호와 동일한 과목번호가 존재합니다.':
                    alert.accept()
                    break

                else:
                    alert.accept()
                    time.sleep(0.5)

            except ElementClickInterceptedException:
                try:
                    time.sleep(0.5)
                    dom = BeautifulSoup(driver.page_source, 'lxml')
                    source = dom.select_one('#captcha_image_t')
                    img = re.findall(r'randomImg/(.+?)\.png', str(source))[0]
                    decoder = base64.b64decode(img)
                    string = decoder.decode('utf-8')

                    driver.find_element_by_id('captcha_answer_u').clear()
                    driver.find_element_by_id('captcha_answer_u').send_keys(string)
                    driver.find_element_by_tag_name('button').click()
                except Exception:
                    continue
        except Exception:
            dom = BeautifulSoup(driver.page_source, 'lxml')
            source = dom.select_one('#captcha_image_t')
            img = re.findall(r'randomImg/(.+?)\.png', str(source))[0]
            decoder = base64.b64decode(img)
            string = decoder.decode('utf-8')

            driver.find_element_by_id('captcha_answer_u').clear()
            driver.find_element_by_id('captcha_answer_u').send_keys(string)
            driver.find_element_by_tag_name('button').click()